### Create CSV from midi sources

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import glob
import os
from tqdm import tqdm
from IPython.display import Image, Audio
import traceback

In [3]:
# parallel
from functools import partial
from pathlib import Path

In [4]:
from data_sources import *
from midi_data import *

In [5]:
path = Path('data/midi')
orig_path = path/'midi_sources'
metapath = path/'metadata'
metapath.mkdir(parents=True, exist_ok=True)

In [6]:
def create_paths(dirname):
    "Standardize midi_source paths"
    dir_path = orig_path/dirname
    json_path = metapath/f'{dirname}_metadata.json'
    csv_path = metapath/f'{dirname}_metadata.csv'
    return dir_path, json_path, csv_path

In [7]:
directories = [x for x in path.iterdir() if x.is_dir()]; directories

[PosixPath('data/midi/metadata'),
 PosixPath('data/midi/midi_sources'),
 PosixPath('data/midi/transcribed_allcomp'),
 PosixPath('data/midi/transposed_old'),
 PosixPath('data/midi/metadata_old')]

### Remove corrupted file - this causes deadlock with music21 processing

In [43]:
Path('data/midi/midi_sources/midiworld/named_midi/NITRO_BRO_-_IT_WONT_DIE.mid').unlink()

### Hooktheory

In [8]:
ht_path, ht_json, ht_csv = create_paths('hooktheory')
ht_song_list = metapath/'hooktheory_key2info.json'

In [9]:
def song_key(s): return '_'.join(s.parts[-3:-1])

In [10]:
ht_key2info = load_json(ht_song_list)

if ht_key2info is None:
    song_info = list((ht_path/'xml').glob('*/*/*/*.json'))
    ht_key2info = {song_key(s):json.load(open(s, 'r')) for s in song_info}
    save_json(ht_key2info, ht_song_list)
    len(song_info)

In [11]:
song_json = list((ht_path/'event').glob('*/*/*/*_key.json')); len(song_json) # using json instead of midi for metadata

19876

In [12]:
def get_ht_midifile(json_file):
    return str(json_file.with_suffix('.mid')).replace('event', 'pianoroll').replace('symbol_', '')

In [13]:
def get_hooktheory_attr(fp):
    song_info = ht_key2info[song_key(fp)]
    song_json = json.load(open(fp, 'r'))
    metadata = song_json['metadata']
    artist = fp.parts[-3]
    title = fp.parts[-2]
    section = fp.name.split('_')[0]
    midi_path = get_ht_midifile(fp)
    
    ht_key = metadata['key']
    ht_mode = metadata['mode']
    if ht_mode is None: ht_mode = 'major'
    ht_offset = keyc_offset(ht_key, ht_mode)
    
    # convert stream here
    metadata = {
        'artist': artist,
        'title': title,
        'section': section,
        'original_path': midi_path,
        'parts': song_info['section'],
        'song_url': song_info['song_url'],
        'genres': song_info['genres'],
        'midi_title': metadata['title'],
        'source': 'hooktheory',
        'ht_bpm': metadata['BPM'],
        'ht_mode': metadata['mode'],
        'ht_key': metadata['key'],
        'ht_offset': ht_offset,
        'ht_time_signature': metadata['beats_in_measure']
    }
    
    return {
        'file_path': midi_path, # midi path not json path
        'metadata': metadata
    }

In [14]:
# sanity check
# hook_out = get_hooktheory_attr(song_json[1000]); hook_out

In [15]:
ht_metadata = parse_midi_dir(song_json, ht_json, meta_func=get_hooktheory_attr, key_func=get_ht_midifile)

Midi Exeption: data/midi/midi_sources/hooktheory/pianoroll/w/weezer/beverly-hills/intro-and-verse_key.mid list index out of range


Midi Exeption: data/midi/midi_sources/hooktheory/pianoroll/w/wolfgang-amadeus-mozart/symphony-no-25-in-g-minor/intro-and-verse_key.mid list index out of range
Midi Exeption: data/midi/midi_sources/hooktheory/pianoroll/y/ylvis/the-fox---what-does-the-fox-say/verse-and-pre-chorus_key.mid list index out of range
Midi Exeption: data/midi/midi_sources/hooktheory/pianoroll/j/jay-z/so-ambitious-feat-pharrel-williams/intro_key.mid list index out of range
Midi Exeption: data/midi/midi_sources/hooktheory/pianoroll/j/johnny-cash/folsom-prison-blues/verse_key.mid list index out of range
Midi Exeption: data/midi/midi_sources/hooktheory/pianoroll/j/jack-johnson/do-you-remember/intro-and-verse_key.mid list index out of range
Midi Exeption: data/midi/midi_sources/hooktheory/pianoroll/j/justin-moore/if-heaven-wasnt-so-far-awat/intro-and-verse_key.mid list index out of range
Midi Exeption: data/midi/midi_sources/hooktheory/pianoroll/j/janis-ian/at-spianorolleen/verse_key.mid File not found or no such fo

Midi Exeption: data/midi/midi_sources/hooktheory/pianoroll/t/tame-impala/pianorollually/intro_key.mid File not found or no such format found for: data/midi/midi_sources/hooktheory/pianoroll/t/tame-impala/pianorollually/intro_key.mid
Midi Exeption: data/midi/midi_sources/hooktheory/pianoroll/t/tame-impala/pianorollually/verse_key.mid File not found or no such format found for: data/midi/midi_sources/hooktheory/pianoroll/t/tame-impala/pianorollually/verse_key.mid
Midi Exeption: data/midi/midi_sources/hooktheory/pianoroll/t/tame-impala/pianorollually/pre-chorus-and-chorus_key.mid File not found or no such format found for: data/midi/midi_sources/hooktheory/pianoroll/t/tame-impala/pianorollually/pre-chorus-and-chorus_key.mid
Midi Exeption: data/midi/midi_sources/hooktheory/pianoroll/t/the-beatles/drive-my-car/verse_key.mid list index out of range
Midi Exeption: data/midi/midi_sources/hooktheory/pianoroll/t/the-beatles/the-end/solo_key.mid list index out of range
Midi Exeption: data/midi/mi

#### Save song_list

In [16]:
ht_metadata = load_json(ht_json)
arr2csv(ht_metadata.values(), ht_csv)
df = pd.read_csv(ht_csv); df.head()

,inferred_offset,inferred_key,instruments,source,seconds,original_path,genres,ht_mode,title,ht_offset,ht_bpm,artist,midi_title,section,parts,song_url,ht_key,time_signature,bpm,ht_time_signature
0,0.0,C major,"Piano,Piano",hooktheory,15.000000,data/midi/midi_sources/hooktheory/pianoroll/w/...,NaN,1.0,yu-gi-oh-theme-song,0,128,wayne-sharpe,yu-gi-oh3,chorus,"intro,chorus",https://www.hooktheory.com/theorytab/view/wayn...,C,4/4,128.0,4
1,-3.0,E- major,"Piano,Piano",hooktheory,11.111111,data/midi/midi_sources/hooktheory/pianoroll/w/...,NaN,1.0,falling-for-you,-3,108,weezer,falling for you intro,intro,"intro,verse,chorus,solo",https://www.hooktheory.com/theorytab/view/weez...,Eb,4/4,108.0,4
2,6.0,E- minor,"Piano,Piano",hooktheory,26.666667,data/midi/midi_sources/hooktheory/pianoroll/w/...,NaN,1.0,falling-for-you,-3,108,weezer,falling for you solo,solo,"intro,verse,chorus,solo",https://www.hooktheory.com/theorytab/view/weez...,Eb,4/4,108.0,4
3,NaN,NaN,NaN,hooktheory,NaN,data/midi/midi_sources/hooktheory/pianoroll/w/...,NaN,NaN,beverly-hills,0,128,weezer,My New Song,intro-and-verse,intro-and-verse,https://www.hooktheory.com/theorytab/view/weez...,C,NaN,NaN,4
4,3.0,A major,"Piano,Piano",hooktheory,13.150685,data/midi/midi_sources/hooktheory/pianoroll/w/...,NaN,1.0,owls,3,146,weebl,Owls - Outro,outro,"verse,outro",https://www.hooktheory.com/theorytab/view/weeb...,A,4/4,146.0,4


In [17]:
df.shape

(19876, 20)

## FreeMidi

In [9]:
fm_path, _, fm_csv = create_paths('freemidi')
fm_dance_path = metapath/f'freemidi_dance_metadata.json'
fm_pop_path = metapath/f'freemidi_pop_metadata.json'
list(fm_path.glob('*'))

[PosixPath('data/midi/midi_sources/freemidi/genre-disco'),
 PosixPath('data/midi/midi_sources/freemidi/genre-pop'),
 PosixPath('data/midi/midi_sources/freemidi/genre-dance-eletric'),
 PosixPath('data/midi/midi_sources/freemidi/genre-punk'),
 PosixPath('data/midi/midi_sources/freemidi/genre-hip-hop-rap'),
 PosixPath('data/midi/midi_sources/freemidi/genre-rock')]

In [19]:
def parse_freemidi_songs(fp, genres=None, source=None):
    name = fp.with_suffix('').name.split(' - ')
    artist = name[0]
    title = name[-1]
    metadata = {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': str(fp),
        'genres': genres,
        'source': source
    }
    return {
        'file_path': fp,
        'metadata': metadata
    }

In [20]:
d_parse_func = partial(parse_freemidi_songs, genres='dance', source='freemidi')
dir_path = fm_path/'genre-dance-eletric'
file_list = get_files(dir_path, extensions=['.mid'], recurse=True)
fm_dance_list = parse_midi_dir(file_list, fm_dance_path, meta_func=d_parse_func)

Midi Exeption: data/midi/midi_sources/freemidi/genre-dance-eletric/Bjork - Glora.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/midi_sources/freemidi/genre-dance-eletric/Tori Amos - Not The Red Baron.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/midi_sources/freemidi/genre-dance-eletric/Gloria Estefan - I'm Not Givin' You Up.mid badly formated midi bytes, got: b'RIFFB\x8c\x00\x00RMIDdata~\x8b\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-dance-eletric/Depeche Mode - Freelove.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/midi_sources/freemidi/genre-dance-eletric/Atomic Kitten - Whole Again.mid badly formated midi bytes, got: b'RIFF\x08K\x00\x00RMIDdata{J\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-dance-eletric/Apol

In [21]:
p_parse_func = partial(parse_freemidi_songs, genres='pop', source='freemidi')
dir_path = fm_path/'genre-pop'
file_list = get_files(dir_path, extensions=['.mid'], recurse=True)
fm_pop_list = parse_midi_dir(file_list, fm_pop_path, meta_func=p_parse_func)

Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Cyndi Lauper - Who Let In The Rain.mid badly formated midi bytes, got: b'0&\xb2u\x8ef\xcf\x11\xa6\xd9\x00\xaa\x00b\xcel?\x13\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Cyndi Lauper - Whats Going On.mid badly formated midi bytes, got: b'0&\xb2u\x8ef\xcf\x11\xa6\xd9\x00\xaa\x00b\xcel?\x13\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Chris De Burgh - Discovery.mid list index out of range
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Michael Jackson - BilliJeans_AI_composed.mid list index out of range
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Sting - A Thousand Years.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/5th Dimension - One Less Bell To Answere.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/midi_sources/freemidi/genre

Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/ABBA - Anthem.mid badly formated midi bytes, got: b'error with file'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Cyndi Lauper - Shattered Dreams.mid badly formated midi bytes, got: b'0&\xb2u\x8ef\xcf\x11\xa6\xd9\x00\xaa\x00b\xcel\xfc0\x02\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Anastacia - Stupid Little Things.mid list index out of range
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Gwen Stefani - Early Winter.mid badly formated midi bytes, got: b'<br />\n<b>Warning</b'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Jon Secada - Just Another Day.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/One Direction - Gotta Be You.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/midi_sources/freemid

Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Kinks - Arthur.mid list index out of range
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Wilson Phillips - Release.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Evanescence - Haunted.mid badly formated midi bytes, got: b'RIFF\x0e[\x00\x00RMIDdata\x01[\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Box Car Racer - Cat Like Thief.mid badly formated midi bytes, got: b'RIFF\xd2\x9b\x00\x00RMIDdata\x9b\x9b\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Vapors - Turning Japanese.mid badly formated midi bytes, got: b'\x00\x00\x0ecmoov\x00\x00\x00lmvhd\x00\x00\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Katy Perry - Hummingbird Heartbeat.mid list index out of range
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Fleetwood Mac - Angel.mid index out of range
Midi Exeption: data/midi/midi_sources/free

In [22]:
fmd = load_json(fm_dance_path)
fmp = load_json(fm_pop_path)
fm_all = list(fmd.values())+list(fmp.values())
arr2csv(fm_all, fm_csv)
df = pd.read_csv(fm_csv); df.head()

,seconds,inferred_offset,midi,artist,instruments,inferred_key,genres,bpm,time_signature,source,title
0,28.732394,-5.0,data/midi/midi_sources/freemidi/genre-dance-el...,Tune Up,NaN,F major,dance,142.0,NaN,freemidi,Bounce
1,85.263158,2.0,data/midi/midi_sources/freemidi/genre-dance-el...,Radiohead,"Piano,Piano",G minor,dance,76.0,4/4,freemidi,Fitter Happier
2,40.000000,-4.0,data/midi/midi_sources/freemidi/genre-dance-el...,Daft Punk,"StringInstrument,StringInstrument,StringInstru...",C# minor,dance,102.0,4/4,freemidi,The Grid
3,NaN,NaN,data/midi/midi_sources/freemidi/genre-dance-el...,Bjork,NaN,NaN,dance,NaN,NaN,freemidi,Glora
4,38.400000,3.0,data/midi/midi_sources/freemidi/genre-dance-el...,Enigma,"Clarinet,Clarinet,Guitar,Guitar,Guitar,Guitar,...",A major,dance,300.0,4/4,freemidi,Wanted


### Gather Cprato

In [10]:
cp_path, cp_json, cp_csv = create_paths('cprato')
list(cp_path.glob('*'))[:5]

[PosixPath('data/midi/midi_sources/cprato/Basto - Again And Again (midi By Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/midi_sources/cprato/The Weeknd ft. Lana Del Rey - Stargirl Interlude  (midi by Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/midi_sources/cprato/Two Steps From Hell - Magic of Love  (midi by Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/midi_sources/cprato/Bermuda Loverz - My Girl (Ladidada) (Rimini Rockaz Radio Edit) (Midi By Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/midi_sources/cprato/Cascada - Everytime We Touch (Midi By Carlo Prato) (www.cprato.com).mid')]

In [24]:
def parse_cprato_songs(fp, genres=None, source=None):
    name = fp.with_suffix('').name.split(' - ')
    artist = name[0]
    title = name[-1].replace('(midi by Carlo Prato) (www.cprato.com)', '')
    metadata = {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': str(fp),
        'genres': genres,
        'source': source
    }
    return {
        'file_path': fp,
        'metadata': metadata
    }

In [25]:
cp_meta = partial(parse_cprato_songs, genres='EDM,inferred', source='cprato')
file_list = get_files(cp_path, extensions=['.mid'], recurse=True)
cp_md = parse_midi_dir(file_list, cp_json, cp_meta)

In [26]:
cp = load_json(cp_json)
arr2csv(cp.values(), cp_csv)
df = pd.read_csv(cp_csv); df.head()

,seconds,inferred_offset,midi,artist,instruments,inferred_key,genres,bpm,time_signature,source,title
0,28.531469,4,data/midi/midi_sources/cprato/Alex Gaudino - C...,Alex Gaudino,NaN,A- major,"EDM,inferred",143.0,NaN,cprato,Calabria (Drunken Monkey Remix) (Midi By Carlo...
1,45.801527,-1,data/midi/midi_sources/cprato/Oh Wonder - All ...,Oh Wonder,"Flute,Flute,Flute,Piano,Piano,Piano,Fretless B...",C# major,"EDM,inferred",131.0,4/4,cprato,All We Do
2,30.447761,0,data/midi/midi_sources/cprato/Martin Garrix & ...,Martin Garrix & Bebe Rexha,NaN,A minor,"EDM,inferred",134.0,4/4,cprato,In The Name Of Love
3,27.200000,1,data/midi/midi_sources/cprato/Andy SVGE - Evol...,Andy SVGE,"Piano,Piano,Piano,Piano,Piano,Piano",B major,"EDM,inferred",150.0,4/4,cprato,Evolving
4,29.142857,-4,data/midi/midi_sources/cprato/DuMonde - See Th...,DuMonde,"Piano,Piano,Piano,Piano,Piano,Piano",C# minor,"EDM,inferred",140.0,4/4,cprato,See The Light


### Gather MidiWorld

In [11]:
mw_path, mw_json, mw_csv = create_paths('midiworld')

In [12]:
def parse_midiworld_songs(fp):
    name = fp.with_suffix('').name.replace('_', ' ').split(' - ')
    artist = name[0]
    title = name[-1]
    metadata = {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': str(fp),
        'genres': 'pop,inferred',
        'source': 'midiworld'
    }
    return {
        'file_path': fp,
        'metadata': metadata
    }

In [44]:
# Warning: if you get a deadlock: 
# PosixPath('data/midi/midi_sources/midiworld/named_midi/NITRO_BRO_-_IT_WONT_DIE.mid') is broken
file_list = get_files(mw_path/'named_midi', extensions=['.mid'], recurse=True)
mw_md = parse_midi_dir(file_list, mw_json, parse_midiworld_songs)

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/fastprogress/fastprogress.py:95: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [45]:
mw = load_json(mw_json)
arr2csv(mw.values(), mw_csv)
df = pd.read_csv(mw_csv); df.head()

,bpm,title,midi,inferred_offset,genres,artist,seconds,time_signature,inferred_key,source,instruments
0,176.0,Drumloop,data/midi/midi_sources/midiworld/named_midi/Ko...,-2.0,"pop,inferred",Kona,1.193182,4/4,B minor,midiworld,Piano
1,127.0,Millenium,data/midi/midi_sources/midiworld/named_midi/TV...,3.0,"pop,inferred",TV Themes,41.929134,4/4,A major,midiworld,"Timpani,Taiko,Violin,Contrabass,Voice"
2,160.0,Looney Tunes,data/midi/midi_sources/midiworld/named_midi/TV...,0.0,"pop,inferred",TV Themes,19.250000,4/4,C major,midiworld,"Marimba,Acoustic Bass,Trombone,StringInstrumen..."
3,108.0,Diddy Kong,data/midi/midi_sources/midiworld/named_midi/Vi...,-2.0,"pop,inferred",Video Game Themes,73.333333,4/4,B minor,midiworld,"Fretless Bass,Harmonica"
4,105.0,NaN,data/midi/midi_sources/midiworld/named_midi/Ji...,-5.0,"pop,inferred",Jigsaw,104.806742,4/4,D minor,midiworld,"StringInstrument,Horn,Trombone,StringInstrumen..."


### Yamaha - piano

In [46]:
ec_path, ec_json, ec_csv = create_paths('ecomp')
ec_song = ec_path/'song_list.json'
list(ec_path.glob('*'))[:5]

[PosixPath('data/midi/midi_sources/ecomp/song_list.json'),
 PosixPath('data/midi/midi_sources/ecomp/2017'),
 PosixPath('data/midi/midi_sources/ecomp/2008'),
 PosixPath('data/midi/midi_sources/ecomp/2006'),
 PosixPath('data/midi/midi_sources/ecomp/2004')]

In [47]:
ec_songs = load_json(ec_path/'song_list.json')

In [48]:
def parse_ecomp_songs(fp):
    song_info = ec_songs[fp.name]
    metadata = {
        'artist': song_info['artist'],
        'title': song_info['title'],
        'midi': str(fp),
        'genres': 'classical',
        'source': 'ecomp'
    }
    return {
        'file_path': fp,
        'metadata': metadata
    }

In [ ]:
file_list = get_files(ec_path, extensions=['.mid'], recurse=True)[:1000]
ec_md = parse_midi_dir(file_list, ec_json, parse_ecomp_songs)

Midi Exeption: data/midi/midi_sources/ecomp/2008/Cui04.MID badly formated midi bytes, got: b'<!DOCTYPE HTML PUBLI'
Midi Exeption: data/midi/midi_sources/ecomp/2008/Cui02.MID badly formated midi bytes, got: b'<!DOCTYPE HTML PUBLI'
Midi Exeption: data/midi/midi_sources/ecomp/2008/Cui06.MID badly formated midi bytes, got: b'<!DOCTYPE HTML PUBLI'
Midi Exeption: data/midi/midi_sources/ecomp/2008/Tan03.MID badly formated midi bytes, got: b'<!DOCTYPE HTML PUBLI'
Midi Exeption: data/midi/midi_sources/ecomp/2008/Tan01.MID badly formated midi bytes, got: b'<!DOCTYPE HTML PUBLI'
Midi Exeption: data/midi/midi_sources/ecomp/2008/Cui05.MID badly formated midi bytes, got: b'<!DOCTYPE HTML PUBLI'
Midi Exeption: data/midi/midi_sources/ecomp/2008/Cui01.MID badly formated midi bytes, got: b'<!DOCTYPE HTML PUBLI'
Midi Exeption: data/midi/midi_sources/ecomp/2008/Cui03.MID badly formated midi bytes, got: b'<!DOCTYPE HTML PUBLI'
Midi Exeption: data/midi/midi_sources/ecomp/2008/Cui08.MID badly formated midi b

In [ ]:
ec = load_json(ec_json)
arr2csv(ec.values(), ec_csv)
df = pd.read_csv(ec_csv); df.head()

### Classic Piano

In [ ]:
clc_path, clc_json, clc_csv = create_paths('classic_piano')
list(clc_path.glob('*'))[:5]

In [ ]:
def parse_classic_songs(fp):
    name = fp.with_suffix('').name.split('_')
    artist = name[0]
    title = ' '.join(name[1:])
    metadata = {
        'artist': artist,
        'title': title,
        'midi': str(fp),
        'genres': 'classical',
        'source': 'classical_piano'
    }
    return {
        'file_path': fp,
        'metadata': metadata
    }

In [ ]:
file_list = get_files(clc_path, extensions=['.mid'], recurse=True)
clc_md = parse_midi_dir(file_list, clc_json, parse_classic_songs)

In [ ]:
clc = load_json(clc_json)
arr2csv(clc.values(), clc_csv)
df = pd.read_csv(clc_csv); df.head()

### Creating CSV

In [ ]:
all_csvs = [ht_csv, fm_csv, cp_csv, mw_csv, ec_csv, clc_csv]
all_dfs = [pd.read_csv(csv) for csv in all_csvs]

In [ ]:
merged_df = pd.concat(all_dfs); merged_df.head()

In [ ]:
[df.shape for df in all_dfs], merged_df.shape

In [ ]:
merged_df.to_csv(metapath/'combined.csv')